In [1]:
import random, math
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn import datasets
import functools
from IPython.display import display, clear_output
random.seed(12345)

Para cada experimento se pasarán ciertos datos extra que serán datos que tendremos en cuenta al momento de experimentar, por ejemplo la página más referenciada y sus valores, o las principales que hacen que nosotros tengamos en cuenta ciertos principios a la hora de evaluar

In [2]:
# Generar txt y guardar instancia
def save_instance(instancia, matriz, solucion, resultado, index, n):
    if(not os.path.isdir(F"instancias/{instancia}")):
        os.mkdir(F"instancias/{instancia}")
                
    with open(F"instancias/{instancia}/{instancia}-{index}-matriz.txt", "w") as f:
        print(n, file=f, end ="\n")
        for i in range(0, len(matriz)):
            for j in range(0, len(matriz[i])):
                print(matriz[i][j], file=f, end=" ")
            print(file=f, end="\n")
            
    with open(F"instancias/{instancia}/{instancia}-{index}-solucion.txt", "w") as f:
        print(n, file=f, end ="\n")
        for i in range(0, len(solucion)):
            print(solucion[i], file=f, end="\n")
            
    with open(F"instancias/{instancia}/{instancia}-{index}-resultado.txt", "w") as f:
        print(n, file=f, end ="\n")
        for i in range(0, len(resultado)):
            print(resultado[i], file=f, end="\n")

In [3]:
def guardar_caso_instancias_enCSV(folder, arr_instancias):
    pd.DataFrame(arr_instancias, columns=["dataset", "archivo-A", "archivo-x", "archivo-b", "n"]).to_csv(F"instancias/{folder}/indice.csv", index=False, header=True)

## Generar x

In [4]:
def generate_x(n=5, nmin = 1, nmax = 100, data_type="int"):
    if(data_type == "int"):
        return np.random.randint(nmin, nmax, size=n)
    else:
        return np.random.rand(n)

### Matrices EDD

In [5]:
def dt_edd(n=5, nmin = 1, nmax = 100, data_type="int"):
    B = np.zeros((n,n))
    if(data_type == "int"):
        B = np.random.randint(nmin, nmax, size=(n,n))
    else:
        B = np.random.rand(n,n)
        
    A = B.copy()
    for i in range(0, n) :        
        suma = 0
        for j in range(0, n) :
            suma = suma + abs(B[i][j])
        A[i][i] = suma
    return A

### Matrices SDP (Simetricas Definidas Positivas)

In [6]:
def dt_sdp(n):
    A = sklearn.datasets.make_spd_matrix(n)    
    return A

### Matrices Random

In [7]:
def dt_random(n=5, nmin = 1, nmax = 100, data_type="int"):
    A = np.zeros((n,n))
    
    if(data_type == "int"):
        A = np.random.randint(nmin, nmax, size=(n,n))
    else:
        A = np.random.rand(n,n)
        
    L = np.tril(A)
    U = np.triu(A)
    
    for i in range(0, n):
        L[i][i] = 1

    M = L @ U
    
    return M

### Matrices Ortogonales

In [8]:
def rvs(n=5, nmin = 1, nmax = 100, data_type="int"):
    random_state = np.random
    H = np.eye(n)
    D = np.ones((n,))
    for n in range(1, n):
        x = random_state.normal(size=(dim-n+1,))
        D[n-1] = np.sign(x[0])
        x[0] -= D[n-1]*np.sqrt((x*x).sum())
        # Householder transformation
        Hx = (np.eye(dim-n+1) - 2.*np.outer(x, x)/(x*x).sum())
        mat = np.eye(dim)
        mat[n-1:, n-1:] = Hx
        H = np.dot(H, mat)
        # Fix the last sign such that the determinant is 1
    D[-1] = (-1)**(1-(dim % 2))*D.prod()
    # Equivalent to np.dot(np.diag(D), H) but faster, apparently
    H = (D*H.T).T
    return H

## Generador de Matrices

#### EDD

In [12]:
caso="edd";
arr_instancias = []
n_values = [10, 20, 30, 40, 50, 100, 250, 500, 1000, 2500]
for i in range(1, len(n_values)):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    
    n = n_values[i]
    A = dt_edd(n, nmin = np.random.randint(10,50), nmax =np.random.randint(51,90), data_type="float")
    x = generate_x(n, nmin = np.random.randint(1,10), nmax =np.random.randint(20,30), data_type="float")
    b = A @ x
    
    save_instance(caso, A, x, b, i, n)
    arr_instancias.append([caso, F"{caso}-{i}-matriz.txt", F"{caso}-{i}-solucion.txt", F"{caso}-{i}-resultado.txt", n])
    
guardar_caso_instancias_enCSV(caso,arr_instancias)

'Instancia: 9'

In [ ]:
caso="edd-tiempos";
arr_instancias = []
n_values = [10, 20, 30, 40, 50, 100, 250, 500, 1000, 2500]
for i in range(1, len(n_values)):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    
    n = n_values[i]
    A = dt_edd(n, nmin = np.random.randint(10,50), nmax =np.random.randint(51,90), data_type="float")
    x = generate_x(n, nmin = np.random.randint(1,10), nmax =np.random.randint(20,30), data_type="float")
    b = A @ x
    
    save_instance(caso, A, x, b, i, n)
    arr_instancias.append([caso, F"{caso}-{i}-matriz.txt", F"{caso}-{i}-solucion.txt", F"{caso}-{i}-resultado.txt", n])
    
guardar_caso_instancias_enCSV(caso,arr_instancias)

In [10]:
caso="sdp";
arr_instancias = []
n_values = [10, 20, 30, 40, 50, 100, 250, 500, 1000]
for i in range(1, len(n_values)):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    
    n = n_values[i]
    A = dt_sdp(n)
    x = generate_x(n, nmin = np.random.randint(1,10), nmax =np.random.randint(20,30), data_type="float")
    b = A @ x
    
    save_instance(caso, A, x, b, i, n)
    arr_instancias.append([caso, F"{caso}-{i}-matriz.txt", F"{caso}-{i}-solucion.txt", F"{caso}-{i}-resultado.txt", n])
    
guardar_caso_instancias_enCSV(caso,arr_instancias)

'Instancia: 8'

In [12]:
caso="random";
arr_instancias = []
n_values = [10, 20, 30, 40, 50, 100, 250, 500, 1000]
for i in range(1, len(n_values)):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    
    n = n_values[i]
    A = dt_random(n, nmin = np.random.randint(1,10), nmax =np.random.randint(20,30), data_type="float")
    x = generate_x(n, nmin = np.random.randint(1,10), nmax =np.random.randint(20,30), data_type="float")
    b = A @ x
    
    save_instance(caso, A, b, x, i, n)
    arr_instancias.append([caso, F"{caso}-{i}-matriz.txt", F"{caso}-{i}-solucion.txt", F"{caso}-{i}-resultado.txt", n])
    
guardar_caso_instancias_enCSV(caso,arr_instancias)

'Instancia: 8'